*Copyright (C) 2021 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

## DNF regimes

The following examples demonstrate the fundamental regimes of
DNFs and their associated instabilities that are the basis for all
macroscopic behavior of architectures.


In [ ]:
from lava.proc.lif.process import LIF
from lava.lib.dnf.connect.connect import connect
from lava.lib.dnf.operations.operations import Weights
from lava.lib.dnf.operations.operations import Convolution
from lava.lib.dnf.inputs.gauss_pattern.process import GaussPattern
from lava.lib.dnf.inputs.rate_code_spike_gen.process import RateCodeSpikeGen

from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps
from lava.proc.monitor.process import Monitor
from lava.proc.monitor.models import PyMonitorModel

from utils import plot_1d

def demonstrate_regime(kernel):
    shape = (15,)
    time_steps = 600

    # Set up spike generator 1
    gauss_pattern_1 = GaussPattern(shape=shape,
                                   amplitude=0,
                                   mean=11.25,
                                   stddev=2.25)
    spike_generator_1 = RateCodeSpikeGen(shape=shape)
    gauss_pattern_1.a_out.connect(spike_generator_1.a_in)

    # Set up spike generator 2
    gauss_pattern_2 = GaussPattern(shape=shape,
                                   amplitude=0,
                                   mean=3.75,
                                   stddev=2.25)
    spike_generator_2 = RateCodeSpikeGen(shape=shape)
    gauss_pattern_2.a_out.connect(spike_generator_2.a_in)

    # DNF with specified kernel
    dnf = LIF(shape=shape, du=409, dv=2047, vth=200)
    connect(dnf.s_out, dnf.a_in, [Convolution(kernel)])

    # Connect spike input to DNF
    connect(spike_generator_1.s_out, dnf.a_in, [Weights(25)])
    connect(spike_generator_2.s_out, dnf.a_in, [Weights(25)])

    # Set up monitors
    monitor_dnf = Monitor()
    monitor_dnf.probe(target=dnf.s_out, num_steps=time_steps)
    monitor_input1 = Monitor()
    monitor_input1.probe(spike_generator_1.s_out, time_steps)
    monitor_input2 = Monitor()
    monitor_input2.probe(spike_generator_2.s_out, time_steps)

    # Run the network and make changes to spike inputs over time
    condition = RunSteps(num_steps=100)
    run_cfg = Loihi1SimCfg(select_tag="fixed_pt",
                           exception_proc_model_map={Monitor: PyMonitorModel})
    spike_generator_1.run(condition=condition, run_cfg=run_cfg)
    gauss_pattern_1.amplitude = 2300
    gauss_pattern_2.amplitude = 2300
    spike_generator_1.run(condition=condition, run_cfg=run_cfg)
    gauss_pattern_1.amplitude = 11200
    gauss_pattern_2.amplitude = 11200
    spike_generator_1.run(condition=condition, run_cfg=run_cfg)
    gauss_pattern_1.amplitude = 2300
    gauss_pattern_2.amplitude = 2300
    spike_generator_1.run(condition=RunSteps(num_steps=200), run_cfg=run_cfg)
    gauss_pattern_1.amplitude = 0
    gauss_pattern_2.amplitude = 0
    spike_generator_1.run(condition=condition, run_cfg=run_cfg)

    # Get probed data from monitors
    data_dnf = monitor_dnf.get_data()[dnf.name][dnf.s_out.name]
    data_input1 = monitor_input1.get_data()[spike_generator_1.name][spike_generator_1.s_out.name]
    data_input2 = monitor_input2.get_data()[spike_generator_2.name][spike_generator_2.s_out.name]

    # Stop the execution of the network
    spike_generator_1.stop()

    # Plot the probed data
    plot_1d(data_dnf,
            data_input1,
            data_input2)

### Detection

This example demonstrates the _detection instability_ of DNFs. The DNF receives spike input at two locations, whose spike rate changes over time. Between time step 100 and 200, the spike rate is not high enough for the DNF to form a peak and it does not spike. When the input spike rate is increased after time step 200, the DNF goes through the _detection instability_, forming a peak and producing spike output. When the input spike rate is lowered back to its previous value, the DNF keeps spiking and its peak remains stable. This demonstrates hysteresis of detection - the DNF stabilizes its decision about detecting an input. Only when the input is completely removed at time step 500 does the peak disappear in the _reverse detection instability_.


In [ ]:
from lava.lib.dnf.kernels.kernels import MultiPeakKernel

detection_kernel = MultiPeakKernel(amp_exc=83, 
                                   width_exc=3.75, 
                                   amp_inh=-70,
                                   width_inh=7.5)

demonstrate_regime(detection_kernel)

### Selection

This example demonstrates the _selection instability_ of a DNF. With spike input at two locations, the DNF makes a selection decision at time step 100, creating a peak at only one of the input locations. That decision remains stable even though two inputs are present simultaneously for the subsequent 100 time steps. The selection switches only when the first input is deactivates at time step 200. It then remains stable on the second input location even as the first input is activated again at time step 300. This demonstrates hysteresis of selection - the DNF stabilizes its selection decision. The same behavior is demonstrated again between time step 400 and 600 for the first input.

Note: The selection is biased/forced here to always be on the input centered on neuron 11 to fit the structure of the input. To achieve this, this input is activated one time step before the other input. If both were activated at the same time step the selection decision would be random. The DNF may then select the "wrong" input and the example would not work.

In [ ]:
from lava.lib.dnf.kernels.kernels import SelectiveKernel

selection_kernel = SelectiveKernel(amp_exc=20, 
                                   width_exc=2.25, 
                                   global_inh=-15)

demonstrate_regime(selection_kernel)

### Memory

This example demonstrates the memory property of DNFs. The input and connectivity is structured like in the example on 'detection' above but here the local excitation is stronger. This leads to the two peaks in the DNF to remain stable even after the input is deactivated in time step 400. The DNF has thus formed a memory of the inputs being present at those locations. Even though such self-sustained peaks are stable without input, they may track present input that changes its location in a graded manner.

In [ ]:
memory_kernel = MultiPeakKernel(amp_exc=32, 
                                width_exc=2.5, 
                                amp_inh=-18, 
                                width_inh=4.5)

demonstrate_regime(memory_kernel)